In [1]:
import pandas as pd
import datetime
import os
import re
from re import search
from modules.modules import openBrowserCrefisa
from shutil import move
import time

In [2]:
from selenium.webdriver.firefox.webdriver import WebDriver
from selenium.webdriver.firefox.service import service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions  as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import *

In [8]:


# classe para baixar relatorio de conciliacao
class conciliation_doenload():

    def __init__(self, driver: WebDriver, date_work = datetime.date, type_report: list = ['Adiantamento', 'Fechamento']):
        self.driver = driver
        self.date_work = date_work
        self.type_report = type_report
        self.path = f'./rel_comissao/{self.date_work.year}/{self.date_work.month}/'
        self.mkdir_rel_financeiro()


    def mkdir_rel_financeiro(self):
        os.makedirs(self.path, exist_ok = True)


    def _move_file(self): 

        path_download = f"./download/"
        path_to_move = self.path
       
        
        os.makedirs(path_to_move, exist_ok = True)
        if os.path.exists(path_download):
            while os.listdir(path_download) == 0:
                continue
            time.sleep(3)
            if self.type_report[0] == 'Adiantamento':
                move(path_download + os.listdir(path_download)[0], path_to_move + f"/Relatorio_financeiro_semanal_{self.date_work}.xls")
            else:        
                move(path_download + os.listdir(path_download)[0], path_to_move + f"/Relatorio_financeiro_mensal_{self.date_work}.xls")


    def report_acess(self):
        driver = self.driver
        try:
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#menu1"))).click()
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#listaMenu1 > li:nth-child(1) > a:nth-child(1)"))).click()
        except ElementClickInterceptedException:
            button = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#menu1")))
            driver.execute_script('arguments[0].click();', button)
            
        
        try:
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtAutenticacaoSenhaFinanceira'))).send_keys('1010')
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button.btn:nth-child(2)'))).click()
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.swal2-confirm'))).click()
        except TimeoutException:
            pass            
        
        

        tabela = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.table-bordered > tbody:nth-child(2)')))
        elements_table = tabela.find_elements(By.CSS_SELECTOR, 'tr')
        
        for i in elements_table:
            print(self.date_work.strftime("%d/%m/%Y"))
            print(i.text)
            if search(str(self.date_work.strftime("%d/%m/%Y")), i.text):
                if self.type_report[0] == 'Adiantamento' and search('Adiantamento', i.text):
                    button = i.find_element(By.CSS_SELECTOR, 'td:nth-child(7) > center:nth-child(1) > div:nth-child(2) > img:nth-child(1)')
                    driver.execute_script('arguments[0].click();', button)
                    i.find_element(By.CSS_SELECTOR, '.open > ul:nth-child(2) > li:nth-child(3) > a:nth-child(1)').click()
                    self._move_file()
                    break
                if self.type_report[0] == 'Fechamento' and search('Fechamento', i.text):
                    buttom = i.find_element(By.CSS_SELECTOR, ' td:nth-child(8) > center:nth-child(1) > div:nth-child(2) > img:nth-child(1)')
                    driver.execute_script('arguments[0].click();', button)
                    i.find_element(By.CSS_SELECTOR, '.open > ul:nth-child(2) > li:nth-child(3) > a:nth-child(1)').click()
                    self._move_file()
                    break
        


In [9]:
teste = conciliation_doenload(driver = openBrowserCrefisa().loguinSendKeys(),
                      date_work = datetime.date(2024, 4, 12),
                      type_report = ['Adiantamento']).report_acess()

12/04/2024
12/04/2024 - 15:30:00
Adiantamento
R$ 4.939,38
GERAL
217102


In [14]:
# classe para conciliar contratos
class conciliation_contracts():

    def __init__(self, date_work: datetime.date, period_search: list = ['Weekly', "monthly"]):
        self.date_work = date_work
        self.root_path = f"./rel_comissao/{date_work.year}/{date_work.month}/"
        self.period_search = period_search[0] 
        # self.conciliation()

   

    # metodo para ler arquivo de comissao consolidada
    def _read_consolidate_comission(self):
        path_to_read = self.root_path + f'Relatorio_financeiro_semanal_{self.date_work}.xls' if self.period_search == 'weekly' else self.root_path + f'Relatorio_financeiro_mensal_{self.date_work}.xls'
        if os.path.exists(path_to_read):
            consolidate_comission = pd.read_html(path_to_read, thousands='.', header=0)[0]
            return consolidate_comission
        else:
            return


      # metdo para ler arquivos de pagamentos antecipados
    def _read_comission_antecipated(self):
        period_search_date = 50 if self.period_search == 'weekly' else 20
        date_to = self.date_work + datetime.timedelta(days = 5)
        date_from = date_to - datetime.timedelta(days = period_search_date)
        range_dates = pd.date_range(start = date_from, end =date_to)
        reports_antecipated = pd.DataFrame()
        for date in range_dates:
            path_to_read_tables = f"../Importados Storm/02 - COMISSÃO/{date.year}/{date.month}-{date.year}"
            os.system('cls')
            # print("Lendo arquivo de comissão do dia: ", date)
            if os.path.exists(path_to_read_tables):
                for file in os.listdir(path_to_read_tables):
                    if not re.search(r'Copia|Cópia|copia|cópia|Ajuste|ajuste|CONSIGNADO|consignado', file):
                            new_data = pd.read_csv(path_to_read_tables + "/" + file, sep = ";", decimal='.')
                            new_data['meta_data'] = file
                            if not new_data.empty:
                                reports_antecipated = pd.concat([reports_antecipated, new_data])

        reports_antecipated.drop_duplicates(inplace = True)
        if not reports_antecipated.empty:
            return reports_antecipated
        else:
            return


    # inserir aqui uma função para carregar os arquivos de pagamentos não realizados, vamos fltrar o consolidate final com ele, e salvar
    # só as que já não estavam
    def 

    # metodo para conferir comissoes - relatório mensal
    def conciliation_monthly(self):
        path_to_save_no_paymet_comission = f'../Comissoes_nao_pagas/{self.date_work.year}/{self.date_work.month}/'
        os.makedirs(path_to_save_no_paymet_comission, exist_ok=True)
        consolidate = self._read_consolidate_comission()
        antecipated = self._read_comission_antecipated()

        antecipated.drop_duplicates(inplace=True)
        consolidate.drop_duplicates(inplace=True)
        
        
        if consolidate is not None and antecipated is not None:

            
            propostas_antecipados = set(antecipated['#ADE#'])
            propostas_consolidados = set(consolidate['Num_Proposta'].values)
            
            diferenca = propostas_consolidados - propostas_antecipados
            
            if not consolidate[consolidate['Num_Proposta'].isin(list(diferenca))].empty:
                print("Foram encontradas comissões não pagas!")
                consolidate_final = consolidate[consolidate['Num_Proposta'].isin(list(diferenca))]
                consolidate_final = consolidate_final.drop_duplicates(subset = ['Num_Proposta'])
                consolidate_final.to_excel(path_to_save_no_paymet_comission + f'comissoes_nao_pagas_{self.date_work}.xlsx', index=False)
                consolidate_final.to_excel(path_to_save_no_paymet_comission + f'somente propostas_{self.date_work}.xlsx', index=False)
                
            else: print("Todas as comissões foram pagas!")
        else: print("Arquivos de comissão consolidada ou comissões antecipadas não foram lidos. Verifique se o arquivo está vazio, e se foram carregados corretamente neste procedimento.")



teste_arquivo = conciliation_contracts(date_work=datetime.date(2024, 4, 8), 
                                       period_search = ['weekly']).conciliation_monthly()


Foram encontradas comissões não pagas!
